# Chapter6 機械学習の基礎

## 50 データの入手、整形

In [2]:
! ls

NewsAggregatorDataset.zip Untitled.ipynb


In [21]:
import pandas as pd

In [22]:
data = pd.read_csv("./NewsAggregatorDataset/newsCorpora.csv",sep="\t",header=None)

In [23]:
data.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')

In [24]:
### column名の追加

In [25]:
data.columns=['ID','TITLE','URL','PUBLISHER','CATEGORY','STORY','HOSTNAME','TIMESTAMP']

In [26]:
data.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [35]:
### 情報源の抽出

In [28]:
publisher={'Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail'}

In [31]:
data = data[data['PUBLISHER'].isin(publisher)]

In [34]:
data['PUBLISHER'].unique()

array(['Reuters', 'Businessweek', 'Huffington Post', 'Daily Mail',
       'Contactmusic.com'], dtype=object)

In [36]:
### ランダムな並び替え

In [37]:
data_shuffled = data.sample(frac=1, random_state=0)

In [38]:
data_shuffled.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
310638,311098,RPT-Fitch Updates EMEA Consumer ABS Rating Cri...,http://in.reuters.com/article/2014/06/24/fitch...,Reuters,b,d3tIMfB2mg-9MZM4G_jGTEiRVl3jM,in.reuters.com,1403633888597
49197,49198,"Gurlitt Wants to Return Nazi-Looted Art, Suedd...",http://www.businessweek.com/news/2014-03-26/gu...,Businessweek,e,dWeC4g3bvX-bI6MKAcxt43Jp7MzSM,www.businessweek.com,1395886231417
410059,410578,"UPDATE 1-Fairfax Financial, CEO probed over po...",http://in.reuters.com/article/2014/08/01/fairf...,Reuters,b,d6s61cXf9rASExMO8inpkRujc9VTM,in.reuters.com,1406929609070
270222,270668,Angelina Jolie - Angelina Jolie Will Not Tight...,http://www.contactmusic.com/story/angelina-jol...,Contactmusic.com,e,d7SbUKanRaD34YMwE64lo09d_qCnM,www.contactmusic.com,1401828902205
301196,301656,Patent Officials Cancel the Washington Redskin...,http://www.businessweek.com/articles/2014-06-1...,Businessweek,b,dXVU3KhKpHSxleMNmc46pQj6e5zwM,www.businessweek.com,1403113194649


In [39]:
### trainとtestにsplit

In [45]:
TRAIN_FILE="./data/train.txt"
TEST_FILE="./data/test.txt"

In [41]:
from sklearn.model_selection import train_test_split

In [43]:
train_data,test_data = train_test_split(data_shuffled,test_size=0.1)

In [46]:
train_data.to_csv(TRAIN_FILE,sep="\t")
test_data.to_csv(TEST_FILE,sep="\t")

In [56]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12006 entries, 70510 to 397190
Data columns (total 8 columns):
ID           12006 non-null int64
TITLE        12006 non-null object
URL          12006 non-null object
PUBLISHER    12006 non-null object
CATEGORY     12006 non-null object
STORY        12006 non-null object
HOSTNAME     12006 non-null object
TIMESTAMP    12006 non-null int64
dtypes: int64(2), object(6)
memory usage: 844.2+ KB


In [57]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 154486 to 240574
Data columns (total 8 columns):
ID           1334 non-null int64
TITLE        1334 non-null object
URL          1334 non-null object
PUBLISHER    1334 non-null object
CATEGORY     1334 non-null object
STORY        1334 non-null object
HOSTNAME     1334 non-null object
TIMESTAMP    1334 non-null int64
dtypes: int64(2), object(6)
memory usage: 93.8+ KB


In [59]:
## 51 特徴量抽出

In [72]:
features=["TITLE","URL","PUBLISHER","STORY","HOSTNAME","TIMESTAMP"]

In [65]:
train_feature = train_data[features]

In [66]:
train_target = train_data["CATEGORY"]

In [67]:
train_target.unique()

array(['b', 'm', 'e', 't'], dtype=object)

In [71]:
train_feature["HOSTNAME"].unique()

array(['in.reuters.com', 'www.reuters.com', 'www.huffingtonpost.com',
       'www.businessweek.com', 'www.contactmusic.com',
       'www.dailymail.co.uk'], dtype=object)